In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [3]:
import spacy

In [4]:
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

C:\Users\bryan\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
nlp.max_length = 11198623

In [6]:
def seperate_punc(doc_text):
    return[token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [7]:
d = read_file(r'C:\Users\bryan\Desktop\Computer Stuff\tensor_flow_class\TF_2_Notebooks_and_Data\06-NLP-and-Text-Data\melville-moby_dick.txt')

In [8]:
tokens = seperate_punc(d)

['chapter',
 '1',
 'loomings',
 'call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore',
 'i',
 'thought',
 'i',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part',
 'of',
 'the',
 'world',
 'it',
 'is',
 'a',
 'way',
 'i',
 'have',
 'of',
 'driving',
 'off',
 'the',
 'spleen',
 'and',
 'regulating',
 'the',
 'circulation',
 'whenever',
 'i',
 'find',
 'myself',
 'growing',
 'grim',
 'about',
 'the',
 'mouth',
 'whenever',
 'it',
 'is',
 'a',
 'damp',
 'drizzly',
 'november',
 'in',
 'my',
 'soul',
 'whenever',
 'i',
 'find',
 'myself',
 'involuntarily',
 'pausing',
 'before',
 'coffin',
 'warehouses',
 'and',
 'bringing',
 'up',
 'the',
 'rear',
 'of',
 'every',
 'funeral',
 'i',
 'meet',
 'and',
 'especially',
 'whenever',
 'my',
 'hypos',
 'get',
 'such

In [9]:
len(tokens)

214712

In [10]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [11]:
' '.join(text_sequences[0])

'chapter 1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to'

In [12]:
' '.join(text_sequences[1])

'1 loomings call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest'

In [13]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [15]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [17]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")
    
    #tokenizer.index_word

158 : chapter
9447 : 1
17527 : loomings
402 : call
42 : me
1043 : ishmael
43 : some
247 : years
659 : ago
140 : never
296 : mind
116 : how
82 : long
787 : precisely
347 : having
113 : little
36 : or
50 : no
1788 : money
6 : in
49 : my
3028 : purse
3 : and
218 : nothing
442 : particular
5 : to


In [19]:
vocabulary_size = len(tokenizer.word_counts)

In [20]:
vocabulary_size

17527

In [21]:
sequences = np.array(sequences)

In [22]:
sequences

array([[  158,  9447, 17527, ...,   218,   442,     5],
       [ 9447, 17527,   402, ...,   442,     5,  1165],
       [17527,   402,    42, ...,     5,  1165,    42],
       ...,
       [  240,   937,   351, ...,  1419,  1313,    74],
       [  937,   351,  1418, ...,  1313,    74,   219],
       [  351,  1418,     3, ...,    74,   219,   222]])

In [23]:
from keras.utils import to_categorical

In [24]:
X = sequences[:,:-1]

In [25]:
y = sequences[:,-1]
y

array([   5, 1165,   42, ...,   74,  219,  222])

In [26]:
y = to_categorical(y,num_classes=vocabulary_size+1)

In [27]:
seq_len = X.shape[1]
seq_len

25

In [28]:
X.shape

(214686, 25)

In [29]:
from keras.models import Sequential

In [30]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [31]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(100,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [32]:
early_stop = EarlyStopping(monitor='val_loss',mode='min', verbose=1, patience=25)

In [33]:
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            438200    
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 100)           50400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 17528)             1770328   
Total params: 2,294,228
Trainable params: 2,294,228
Non-trainable params: 0
_________________________________________________________________


In [34]:
from pickle import dump,load

In [ ]:
model.fit(X,y,batch_size=128, epochs=350, verbose=1, callbacks=[early_stop])

C:\Users\bryan\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/350
214686/214686 [==============================] - 317s 1ms/step - loss: 7.0048 - accuracy: 0.0686
Epoch 2/350
214686/214686 [==============================] - 349s 2ms/step - loss: 6.6093 - accuracy: 0.0839
Epoch 3/350
214686/214686 [==============================] - 342s 2ms/step - loss: 6.3759 - accuracy: 0.0944
Epoch 4/350
214686/214686 [==============================] - 386s 2ms/step - loss: 6.2199 - accuracy: 0.1051
Epoch 5/350
214686/214686 [==============================] - 356s 2ms/step - loss: 6.0953 - accuracy: 0.1118
Epoch 6/350
214686/214686 [==============================] - 350s 2ms/step - loss: 5.9877 - accuracy: 0.1153
Epoch 7/350
214686/214686 [==============================] - 346s 2ms/step - loss: 5.8917 - accuracy: 0.1182
Epoch 8/350
214686/214686 [==============================] - 337s 2ms/step - loss: 5.8032 - accuracy: 0.1205
Epoch 9/350
214686/214686 [==============================] - 333s 2ms/step - loss: 5.7184 - accuracy: 0.1233
Epoch 10/350
214686

In [ ]:
model.save('moby_dick.h5')

In [ ]:
dump(tokenizer,open('my_tokenizer','wb'))

In [ ]:
from keras.preprocessing import sequence

In [ ]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = []
    
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len, truncating='pre')
        
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
        pred_word = tokenizer.index_word[pred_word_ind]
        
        input_text += ' '+pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [ ]:
text_sequences[0]

In [ ]:
import random

random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [ ]:
random_seed_text = text_sequences[random_pick]

In [ ]:
random_seed_text

In [ ]:
seed_text = ' '.join(random_seed_text)

In [ ]:
seed_text

In [ ]:
generate_text(model, tokenizer, seq_len, seed_text=seed_text, num_gen_words=100)